In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
#from linformer import Linformer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install einops
%pip install wandb

     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 163kB 52.8MB/s 
     |████████████████████████████████| 133kB 57.6MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=ddb9b55598914816a87b9f1273a5ab8a4050fd413609a8fee2478b6e498aa4de
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=49d12ebf13bef885781a5b9d9c82773b7d40d650fb94d2a878a3bcc8ffcac6c0
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [4]:
%cd drive/MyDrive/Colab Notebooks/ViT

/content/drive/MyDrive/Colab Notebooks/ViT


In [5]:
from model.vit import ViT
from model.t2t import T2TViT, RearrangeImage
from dataloader import set_transforms
from utils.utils import seed_everything

seed_everything(42)

In [6]:
import torch.backends.cudnn as cudnn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

Use CUDA: True


In [7]:
config={
    "epochs": 20, 
    "batch_size": 64,
    "lr" : 3e-5,
    "gamma" : 0.7,
    "image_size" : 32,
    "patch_size" : 16,
    "num_classes" : 10,
    "dim" : 128,
    "depth" : 12,
    "heads" : 8,
    "mlp_dim" : 1024,
    "channels" : 3,
    "dropout" : 0.,
    "emb_dropout": 0.
    }

# Training settings
import wandb
wandb.init(config=config, project="Cifar10_ViT", entity='NoguNogu',name='T2T')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
trainloader, testloader = set_transforms(
    config['image_size'],
    config['batch_size'],
    name='Cifar10')._set_transforms()

In [14]:
#T2TはPatch_sizeの指定がないことに注意
#nn.Unfoldの計算：https://pytorch.org/docs/stable/generated/torch.nn.Unfold.html
model = T2TViT(
    dim = config['dim'],
    image_size = config['image_size'],
    num_classes= config['num_classes'],
    channels = config['channels'],
    depth = config['depth'],
    heads = config['heads'],
    mlp_dim = config['mlp_dim'],
    t2t_layers = ((3, 2), (3, 2), (3, 2))
    ).to(device)

In [15]:
from torchsummary import summary
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
          Identity-1            [-1, 3, 32, 32]               0
            Unfold-2              [-1, 27, 256]               0
         Rearrange-3              [-1, 256, 27]               0
         LayerNorm-4              [-1, 256, 27]              54
            Linear-5              [-1, 256, 81]           2,187
           Softmax-6          [-1, 1, 256, 256]               0
          Identity-7              [-1, 256, 27]               0
         Attention-8              [-1, 256, 27]               0
           PreNorm-9              [-1, 256, 27]               0
        LayerNorm-10              [-1, 256, 27]              54
           Linear-11              [-1, 256, 27]             756
             GELU-12              [-1, 256, 27]               0
          Dropout-13              [-1, 256, 27]               0
           Linear-14              [-1, 

In [16]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=config['gamma'])

In [17]:
# 3. Log gradients and model parameters
wandb.watch(model, log_freq=100)
for epoch in range(config['epochs']):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(trainloader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in testloader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += val_loss / len(testloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    # 4. Log metrics to visualize performance
    wandb.log({
        "Epoch": epoch+1,
        "loss": epoch_loss,
        "acc": epoch_accuracy,
        "val_loss" : epoch_val_loss,
        "val_acc": epoch_val_accuracy
        })


Epoch : 1 - loss : 1.9409 - acc: 0.2676 - val_loss : 1.8588 - val_acc: 0.3051




Epoch : 2 - loss : 1.7975 - acc: 0.3348 - val_loss : 1.7729 - val_acc: 0.3464




Epoch : 3 - loss : 1.7222 - acc: 0.3725 - val_loss : 1.6691 - val_acc: 0.3939




Epoch : 4 - loss : 1.6534 - acc: 0.3984 - val_loss : 1.6258 - val_acc: 0.4099




Epoch : 5 - loss : 1.6047 - acc: 0.4175 - val_loss : 1.5709 - val_acc: 0.4303



...


Epoch : 16 - loss : 1.2668 - acc: 0.5491 - val_loss : 1.3034 - val_acc: 0.5329




Epoch : 17 - loss : 1.2461 - acc: 0.5573 - val_loss : 1.2721 - val_acc: 0.5456




Epoch : 18 - loss : 1.2273 - acc: 0.5652 - val_loss : 1.2511 - val_acc: 0.5574




Epoch : 19 - loss : 1.2026 - acc: 0.5731 - val_loss : 1.2347 - val_acc: 0.5590




Epoch : 20 - loss : 1.1737 - acc: 0.5839 - val_loss : 1.2040 - val_acc: 0.5711



In [18]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu May  6 13:56:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    55W / 250W |  15537MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------